In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models
from skimage.io import imread
from skimage.transform import resize
import cv2


class GradCamModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.gradients = None
        self.tensorhook = []
        self.layerhook = []
        self.selected_out = None
        
        #PRETRAINED MODEL
        self.pretrained = models.resnet50(pretrained=True)
        #self.pretrained = models.alexnet(pretrained=True)
        #self.pretrained = models.mobilenet_v2(pretrained=True)
        

        self.layerhook.append(self.pretrained.layer3.register_forward_hook(self.forward_hook()))

        for name, param in self.pretrained.named_parameters():
            param.requires_grad = True
    
    def activations_hook(self,grad):
        self.gradients = grad

    def get_act_grads(self):
        return self.gradients

    def forward_hook(self):
        def hook(module, inp, out):
            self.selected_out = out
            self.tensorhook.append(out.register_hook(self.activations_hook))
        return hook

    def forward(self,x):
        out = self.pretrained(x)
        return out, self.selected_out


gcmodel = GradCamModel()#.to('cuda:0')
img = imread('tiger.jfif') #'bulbul.jpg'
img = resize(img, (224,224), preserve_range = True)
img = np.expand_dims(img.transpose((2,0,1)),0)
img /= 255.0
mean = np.array([0.485, 0.456, 0.406]).reshape((1,3,1,1))
std = np.array([0.229, 0.224, 0.225]).reshape((1,3,1,1))
img = (img - mean)/std
inpimg = torch.from_numpy(img).to(torch.float32)

out, acts = gcmodel(inpimg)
acts = acts.detach().cpu()

loss = nn.CrossEntropyLoss()(out,torch.from_numpy(np.array([600])))#.to('cuda:0'))
loss.backward()

grads = gcmodel.get_act_grads().detach().cpu()
pooled_grads = torch.mean(grads, dim=[0,2,3]).detach().cpu()

for i in range(acts.shape[1]):
    acts[:,i,:,:] += pooled_grads[i]

heatmap_j = torch.mean(acts, dim = 1).squeeze()
heatmap_j_max = heatmap_j.max(axis = 0)[0]
heatmap_j /= heatmap_j_max
heatmap_j = resize(heatmap_j,(224,224),preserve_range=True)

cmap = mpl.cm.get_cmap('jet', 256)
heatmap_j2 = cmap(heatmap_j,alpha = 0.2)
fig, axs = plt.subplots(1,1,figsize = (5,5))
axs.imshow((img*std+mean)[0].transpose(1,2,0))
axs.imshow(heatmap_j)
plt.show()

for h in gcmodel.layerhook:
    h.remove()
for h in gcmodel.tensorhook:
    h.remove()
    
    #Alexnet vgg16 - features
    #resnet50 - layer 3

AttributeError: 'torch.dtype' object has no attribute 'char'

In [7]:
m,n = img.shape[::2]
print(m)
print(n)
data_new = img.reshape(n,n,-1)

1
224


In [8]:
heatmap_j.shape

(224, 224)

In [9]:
fi = cv2.addWeighted(np.float32(heatmap_j), 0.7, img, 0.3, 0)

error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-oe0iat4a/opencv/modules/core/src/arithm.cpp:669: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'


In [50]:
import numpy as np
from cv2 import cv2
from skimage import exposure 
...

capture = cv2.VideoCapture(...)
while True:
    ret, frame = capture.read()

    if ret:
        #resize original frame
        frame = cv2.resize(frame, (224, 224)) 

        #get color map
        cam = getMap(frame)
        map_img = exposure.rescale_intensity(cam, out_range=(0, 255))
        map_img = np.uint8(map_img)
        heatmap_img = cv2.applyColorMap(map_img, cv2.COLORMAP_JET)

        #merge map and frame
        fin = cv2.addWeighted(heatmap_img, 0.5, frame, 0.5, 0)

        #show result
        cv2.imshow('frame', fin)

TypeError: Argument 'index' is required to be an integer

In [56]:
image = cv2.imread('/Users/nihalmathew/Downloads/NEU-CA-main/Spring2022/DV/Project/image_classification/tiger.jfif', 1)
gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [18]:
heatmap_img = cv2.applyColorMap(gray_img, cv2.COLORMAP_JET)

In [58]:
gray_img.shape

(183, 275)